## Labelisation des données 

Ce notebook a pour objectif de determiner la classe de chaque joueur afin de créer un modèle de classification binaire.


Paul Paquin, Charlène Petit, Alexandre Ly, Naziz Ismail

## Imports

In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
import random as rd

In [3]:
# je suppose que data_players contient nom joueur performance ainsi que la désiralibilité pour chacun des clusters 
# de plus une variable etat: T ( transféré) P(prété) R(rien)
data_player_17_18 = pd.read_csv("Final_table_special_17_18.csv",sep=",", engine="python",encoding = 'UTF-8')
data_player_18_19 = pd.read_csv("Final_table_special_18_19.csv",sep=",", engine="python",encoding = 'UTF-8')
evol_perf= pd.read_csv('Evolution_performance.csv')

In [4]:
list_players_transf_18=data_player_17_18.loc[data_player_17_18['attribut']=='T']['Name'].tolist()

In [5]:
evol_transf=evol_perf.loc[evol_perf['Name'].isin(list_players_transf_18)]
list_useful_transfers=evol_transf.loc[evol_transf['Performance']>=0]['Name'].tolist()
evol_transf['Performance'].mean()

44.04773460713649

In [7]:
data_player_17_18.loc[~data_player_17_18['Name'].isin(list_useful_transfers)]=data_player_17_18.loc[~data_player_17_18['Name'].isin(list_useful_transfers)].replace({'attribut':'T'},value='R')

## Fonctions auxiliaires

In [8]:
global clusters
clusters = ['000','100','101','110','111','001','010','011']

def distance(player1,player2): #calcul la distance entre 2 points
    return np.sqrt((player1[0]-player2[0])**2 + (player1[1]-player2[1])**2)
    
    
def nuage(data,cluster): #supprime les autres distances
    df = data.copy()
    for i in clusters:
        if i != cluster:
            df.drop(columns=[i])
    return df 

def K_neighbours(x,playersList,dic_distance,K): #determine les K proches voisins de x
        L_neighbours = []
        L_distances = []
        result = [] # Liste de couple d'individus et de distances
        mini_dic_distances = {} #cle : distance valeur : individu
        for i in playersList:
            mini_dic_distances[dic_distance[(x,i)]] = i 
            L_distances.append(dic_distance[(x,i)])
            
        L_distances.sort()
        L_neighbours_distance = L_distances[1:K+1]  #on enlève le 0
        
        for i in L_neighbours_distance:
            individu = mini_dic_distances[dic_distance[(x,mini_dic_distances[i])]]
            result.append([individu,dic_distance[(x,mini_dic_distances[i])]])
    
        return result #renvoie les K individus les plus proches liste de listes 
            # chacune des listes contient [individu, distance au centre]

In [4]:
def labelisation_data(data_players,cluster,K): #K le nombres de proches voisins// renvoie une table contenant le label de chaque
    cloud = nuage(data_players,cluster)                                                                                  #joueur
    nb_lines = cloud.shape[0]
    dic_players = {} # cle : nom joueur, valeur : (performance,desirabilite)
    dic_neighbours = {} # cle : nom joueur, valeur : (liste_voisins,distance au centre) 
    dic_distance = {} # cle : (nom joueur1, nom joueur2), valeur : distance entre joueur1 et joueur2
    dic_label = {} #cle : nom joueur valeur : 1 ou 0
    # Listes 
    playersList = cloud['Name'].tolist()
    centersList_T = [] # liste des joueurs transférés
    centersList_P = [] # liste des joueurs prêtés
    centerList = [] 
    
    for i in range(nb_lines):
        player = cloud.iloc[i]
        joueur = player['Name']
        dic_players[joueur] = [player['Performance'],player[cluster]]
        
        if player['attribut'] == 'T':
            centersList_T.append(joueur) 
        elif player['attribut'] == 'P':
            centersList_P.append(joueur)
            
    dic_players_tool = dic_players.copy()
    
    centerList = centersList_T + centersList_P
    
    for joueur1 in centerList:
        for joueur2 in dic_players_tool.keys():
            dic_distance[(joueur1,joueur2)] = distance(dic_players_tool[joueur1],dic_players_tool[joueur2])
            
    
    for centre in centersList_T:
                dic_neighbours[centre] = K_neighbours(centre,playersList,dic_distance,K)
            
    for centre in centersList_P:    
                dic_neighbours[centre] = K_neighbours(centre,playersList,dic_distance,K-3)      
                
    P = [key for key in dic_neighbours.keys()]            
    #print(dic_neighbours[P[5]])
    #supression des intersections entre K-voisins de chaque centre
    #dic_neighbours = sup_intersection(dic_neighbours,dic_distance)           
    
    #print(dic_neighbours)
    
    for player in playersList:
        dic_label[player] = 0
    
    for center in dic_neighbours.keys():
        dic_label[center] = 1
        for list_center in dic_neighbours[center]:
            dic_label[list_center[0]] = 1
    
    final_list = []
    
    for i in range(nb_lines):
        player = cloud.iloc[i]
        joueur = player['Name']
        final_list.append(dic_label[joueur])
    
    data_players["label "+cluster] = final_list
    
    return data_players

In [9]:
for cluster in clusters:
    data_players_result = labelisation_data(data_player_17_18,cluster,6)


In [10]:
data_players_result

,Equipe,Age,Position 1,VM,000,100,101,110,111,001,...,Allant à,Name,label 000,label 100,label 101,label 110,label 111,label 001,label 010,label 011
0,AC Milan,0.130435,MFC,50000.0,52.084574,74.205775,79.831912,34.502522,47.871807,47.334398,...,NaN,Emanuele Torrasi,1,1,1,1,1,1,1,1
1,AC Milan,0.695652,DMC,20000000.0,42.652218,69.217421,77.207300,41.365161,57.378609,42.653919,...,NaN,Lucas Biglia,1,1,1,1,1,1,1,1
2,AC Milan,0.652174,DC,45000000.0,48.898795,72.647890,51.033987,61.623865,51.214983,48.907455,...,Juventus,Leonardo Bonucci,1,1,1,1,1,1,1,1
3,AC Milan,0.739130,MFC,3500000.0,36.464490,65.846273,75.466815,45.787305,63.513518,34.679849,...,NaN,Riccardo Montolivo,1,1,1,1,1,1,1,1
4,AC Milan,0.608696,FW,20000000.0,54.799202,37.184064,45.191345,32.594654,45.220277,49.539492,...,Atletico Madrid,Nikola Kalinic,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,Wolfsburg,0.434783,MFLR,3000000.0,78.975066,45.916795,21.079088,15.100778,20.977107,69.121517,...,NaN,Renato Steffen,0,0,0,0,0,0,0,0
2070,Wolfsburg,0.391304,AMC,10000000.0,85.109562,12.317767,14.933080,10.690819,14.861740,74.093665,...,NaN,Yunus Malli,0,0,0,0,0,0,0,0
2071,Wolfsburg,0.391304,DC,10000000.0,85.109562,91.936676,14.933080,88.880488,14.861740,85.102625,...,Schalke 04,Jeffrey Bruma,1,1,1,1,1,1,1,1
2072,Wolfsburg,0.173913,FW,6500000.0,67.678026,26.663463,32.353602,23.261377,32.286694,59.970538,...,NaN,Landry Dimata,0,0,0,0,0,0,0,0


In [11]:
data_players_result.to_csv('Labelised_special_17_18.csv')

## Annexes

In [ ]:
# fonction d'optimisation obsolète

In [ ]:
def sup_intersection(dic_voisins,dic_distance): # à arranger pour l'optimisation pour le multiclasse
    
    dic_close_neighbours_removed = dic_voisins.copy() 
    
    for center1 in dic_close_neighbours_removed.keys():
        for center2 in dic_close_neighbours_removed.keys():
            if center1 != center2:
                print('len(center1)',len(center1))
                print('len(center2)',len(center2))
                #comparer le max distance à la distance entre les 2 centres si ce n'est pas supérieur aux 2 distances, chercher les voisins en communs
                if (center1[-1][1] > dic_distance[(center1,center2)]) and (center2[-1][1] > dic_distance[(center1,center2)]): 
                    #après validation de la condition necessaire, on supprime les elements en commun des 2 listes
                    for list1 in  dic_close_neighbours_removed[center1]:
                        for list2 in dic_close_neighbours_removed[center2]:
                            if list1[0] == list2[0]:
                                if list1[1] < list2[1]:
                                    dic_close_neighbours_removed[center2].remove(list2)
                                else:
                                    dic_close_neighbours_removed[center1].remove(list1)
                
    return dic_close_neighbours_removed 